In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.


In [2]:
excel_path = "/Users/AlexanderLarsson/documents/VSCode/test_indata.xlsx"
indata_sheet = "test_case_2_whNBD_NBD"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN,7
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.1,15
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.2,6
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.3,5
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.4,4
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.5,9
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.6,8
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.7,7
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.8,13
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.9,12


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
Q_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Q").to_numpy())
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
demand_mean_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
demand_stdev_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))
R_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Reorder point").to_numpy()
R_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Reorder point").to_numpy())


Calculating lead time demand mean and std dev for RDC

In [4]:
rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
demand_mean_warehouse = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = demand_mean_warehouse
demand_stdev_warehouse = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = demand_stdev_warehouse

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.036479,7,100.0,20.364792
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,15,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,6,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,5,NaN,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,4,NaN,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,9,NaN,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,8,NaN,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,7,NaN,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,13,NaN,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,12,NaN,NaN


Calculating lead times approximations for dealers

In [5]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_warehouse,R_warehouse,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
L_dealer_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = L_dealer_arr
Ldemand_dealer_mean_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = Ldemand_dealer_mean_arr
Ldemand_dealer_stdev_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] = Ldemand_dealer_stdev_arr
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Waiting time,Lead time
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.036479,7,100.000000,20.364792,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,15,10.125786,11.138364,0.125786,10.125786
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,6,3.125786,3.750943,0.125786,3.125786
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,5,2.125786,2.763521,0.125786,2.125786
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,4,1.125786,1.576100,0.125786,1.125786
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,9,4.125786,6.188679,0.125786,4.125786
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,8,3.125786,5.001257,0.125786,3.125786
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,7,2.125786,3.613836,0.125786,2.125786
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,13,5.125786,9.226414,0.125786,5.125786
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,12,4.125786,7.838993,0.125786,4.125786


Calculating fill rate for RDC

In [6]:
#Need fill rate calculations for gamma in service_level_computations


if wh_dist == "Normal":
    outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_normal_demand(R_warehouse, Q_warehouse, demand_mean_warehouse, demand_stdev_warehouse)
#elif wh_dist == "Gamma":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = #add here
#elif wh_dist == "NBD":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_compound_poisson_demand(demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse), IL_prob_array_discrete_positive(R_warehouse, Q_warehouse, demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse)))

outdataDF


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Waiting time,Lead time,Fill rate
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.036479,7,100.000000,20.364792,NaN,NaN,0.923414
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,15,10.125786,11.138364,0.125786,10.125786,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,6,3.125786,3.750943,0.125786,3.125786,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,5,2.125786,2.763521,0.125786,2.125786,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,4,1.125786,1.576100,0.125786,1.125786,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,9,4.125786,6.188679,0.125786,4.125786,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,8,3.125786,5.001257,0.125786,3.125786,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,7,2.125786,3.613836,0.125786,2.125786,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,13,5.125786,9.226414,0.125786,5.125786,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,12,4.125786,7.838993,0.125786,4.125786,NaN


Calculating fill rates for dealers

In [7]:

fill_rate_dealer_arr = []

for i in range(0, len(demand_type_arr)):
    if demand_type_arr[i] == "Normal":
        fill_rate_dealer_arr.append(fill_rate_normal_demand(R_dealer_arr[i], Q_dealer_arr[i], Ldemand_dealer_mean_arr[i], Ldemand_dealer_stdev_arr[i]))
    elif demand_type_arr[i] == "Poisson":
        fill_rate_dealer_arr.append(fill_rate_poisson_demand(IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_arr_poisson(L_dealer_arr[i], Ldemand_dealer_mean_arr[i]))))
    elif demand_type_arr[i] == "NBD":
        print(f"dealer mean arr: {Ldemand_dealer_mean_arr[i]}")
        demand_prob_array = demand_prob_arr_negative_binomial(L_dealer_arr[i], demand_mean_arr[i], demand_stdev_arr[i]**2)
        IL_Prob = IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_array)
        fill_rate_NBD = fill_rate_compound_poisson_demand(demand_prob_array, IL_Prob)
        fill_rate_dealer_arr.append(fill_rate_NBD)

outdataDF.loc[outdataDF["Type"]== "Dealer", "Fill rate"] = fill_rate_dealer_arr
outdataDF

dealer mean arr: 10.125785739210206
dealer mean arr: 3.125785739210206
dealer mean arr: 2.125785739210206
dealer mean arr: 1.1257857392102057
dealer mean arr: 4.1257857392102055
dealer mean arr: 3.125785739210206
dealer mean arr: 2.125785739210206
dealer mean arr: 5.1257857392102055
dealer mean arr: 4.1257857392102055
dealer mean arr: 3.125785739210206


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Waiting time,Lead time,Fill rate
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.036479,7,100.000000,20.364792,NaN,NaN,0.923414
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,15,10.125786,11.138364,0.125786,10.125786,0.788575
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,6,3.125786,3.750943,0.125786,3.125786,0.931321
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,5,2.125786,2.763521,0.125786,2.125786,0.951153
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,4,1.125786,1.576100,0.125786,1.125786,0.970870
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,9,4.125786,6.188679,0.125786,4.125786,0.917895
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,8,3.125786,5.001257,0.125786,3.125786,0.933096
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,7,2.125786,3.613836,0.125786,2.125786,0.948729
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,13,5.125786,9.226414,0.125786,5.125786,0.919870
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,12,4.125786,7.838993,0.125786,4.125786,0.930075
